In [22]:
!pip install pandas scikit-learn matplotlib nbformat

!pip install openai


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [23]:
import os
os.environ['OPENAI_API_KEY'] = input("Paste your OpenAI API key here: ").strip()


Paste your OpenAI API key here:   sk-proj-odxZUP8JEQaQhikRJFnBfLkGQjMI4lOL_6rla-ofsgsn42tyTKJrJ8Sr-Sh7kQp627XDwv36T3BlbkFJ0FVaPY6LLq9eq7ztG067cDdDu2c4Nc-yhmEErVmpGsmL8y5twNxJ9hul1e92sOlEiQGMIEeCUA


In [26]:
import pandas as pd
products = [
    {'name': 'Boho Dress', 'desc': 'Flowy maxi dress in earthy tones, tassels and relaxed fit for festival boho vibes', 'tags': ['boho','festival','flowy']},
    {'name': 'Urban Bomber', 'desc': 'Cropped bomber jacket, sleek lines, reflective accents for energetic urban chic', 'tags': ['urban','chic','energetic']},
    {'name': 'Cozy Knit Sweater', 'desc': 'Soft oversized knit, neutral color, perfect for cozy hygge days', 'tags': ['cozy','casual','warm']},
    {'name': 'Minimal White Tee', 'desc': 'Crisp white tee, clean silhouette, versatile staple for minimal looks', 'tags': ['minimal','basics','clean']},
    {'name': 'Sport Luxe Leggings', 'desc': 'High-waisted leggings with a sheen, athleisure for active city mornings', 'tags': ['sport','athleisure','energetic']},
    {'name': 'Vintage Denim Jacket', 'desc': 'Worn-in denim with patch details for retro-casual vibes', 'tags': ['vintage','retro','casual']},
    {'name': 'Silk Slip Dress', 'desc': 'Bias-cut silk slip in jewel tones for evening glam and effortless elegance', 'tags': ['glam','evening','elegant']},
    {'name': 'Plaid Wool Coat', 'desc': 'Structured wool coat with bold plaid — statement cold-weather piece', 'tags': ['statement','warm','structured']},
]
df = pd.DataFrame(products)
df.index.name = 'id'
df.head()


,name,desc,tags
id,,,
0,Boho Dress,"Flowy maxi dress in earthy tones, tassels and ...","[boho, festival, flowy]"
1,Urban Bomber,"Cropped bomber jacket, sleek lines, reflective...","[urban, chic, energetic]"
2,Cozy Knit Sweater,"Soft oversized knit, neutral color, perfect fo...","[cozy, casual, warm]"
3,Minimal White Tee,"Crisp white tee, clean silhouette, versatile s...","[minimal, basics, clean]"
4,Sport Luxe Leggings,"High-waisted leggings with a sheen, athleisure...","[sport, athleisure, energetic]"


In [38]:
!pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable


In [39]:
import openai
print(" OpenAI package loaded successfully!")


 OpenAI package loaded successfully!


In [40]:
!pip install openai --quiet


In [41]:
import openai
print(" OpenAI package loaded successfully!")


 OpenAI package loaded successfully!


In [42]:
import sys
print(sys.executable)


C:\Users\arrkr\py310_env\Scripts\python.exe


In [33]:
!C:\Users\arrkr\py310_env\Scripts\python.exe -m pip install --upgrade openai


In [34]:
!C:\Users\arrkr\py310_env\Scripts\python.exe -m pip install --upgrade pip


In [35]:
!C:\Users\arrkr\py310_env\Scripts\python.exe -m pip install --upgrade openai


In [36]:
import openai
print(" OpenAI package loaded successfully!")


 OpenAI package loaded successfully!


In [37]:

import os, time, numpy as np, openai


client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_openai_embeddings(texts, model="text-embedding-3-small", batch_size=5, sleep_sec=0.5):
    """Return numpy array of embeddings for each text."""
    all_embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        response = client.embeddings.create(model=model, input=batch)
        for item in response.data:
            all_embs.append(item.embedding)
        time.sleep(sleep_sec)
    return np.array(all_embs)


corpus = df["desc"].tolist()
print("Creating embeddings for", len(corpus), "products...")
product_embs = get_openai_embeddings(corpus)
print(" Done! Shape:", product_embs.shape)


np.save("product_embs.npy", product_embs)


Creating embeddings for 8 products...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [43]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

print(" Using local TF-IDF vectors (free, offline demo)")


vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=256)
corpus = df["desc"].tolist()
product_embs = vectorizer.fit_transform(corpus).toarray()

print(" Done! Shape:", product_embs.shape)


 Using local TF-IDF vectors (free, offline demo)
 Done! Shape: (8, 155)


In [44]:

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

def query_top_k(query_text, k=3, threshold=0.3):
    
    qv = vectorizer.transform([query_text]).toarray()
    sims = cosine_similarity(qv, product_embs)[0]
    
    
    results = df.copy()
    results["similarity_score"] = sims
    
  
    results = results.sort_values("similarity_score", ascending=False).reset_index(drop=True)
    

    if results.iloc[0]["similarity_score"] < threshold:
        print(" No strong match found! Try another vibe keyword.")
        print(" Suggestions: 'cozy casual', 'night out glam', 'urban streetwear'")
    else:
        print(f" Top {k} matches for vibe: '{query_text}'")
    

    display(results.head(k)[["name", "desc", "similarity_score"]])

query_top_k("energetic urban chic")


 Top 3 matches for vibe: 'energetic urban chic'


,name,desc,similarity_score
0,Urban Bomber,"Cropped bomber jacket, sleek lines, reflective...",0.497528
1,Boho Dress,"Flowy maxi dress in earthy tones, tassels and ...",0.000000
2,Cozy Knit Sweater,"Soft oversized knit, neutral color, perfect fo...",0.000000


In [45]:

import time
from IPython.display import display

queries = [
    "energetic urban chic",
    "cozy hygge sweater",
    "evening glam slip dress"
]

records = []
for q in queries:
    print(" Query:", q)
    start = time.perf_counter()
    qv = vectorizer.transform([q]).toarray()
    sims = cosine_similarity(qv, product_embs)[0]
    df_copy = df.copy()
    df_copy["score"] = sims
    df_copy = df_copy.sort_values("score", ascending=False).reset_index(drop=True)
    latency = (time.perf_counter() - start) * 1000  # ms
    top3 = df_copy.head(3)
    display(top3[["name", "desc", "score"]])
    good_matches = (top3["score"] > 0.3).sum()
    records.append({"query": q, "good_matches": good_matches, "latency_ms": latency})
    print("Good matches:", good_matches, "| Latency:", round(latency, 2), "ms")
    print("-" * 80)


results_df = pd.DataFrame(records)
display(results_df)


 Query: energetic urban chic


,name,desc,score
0,Urban Bomber,"Cropped bomber jacket, sleek lines, reflective...",0.497528
1,Boho Dress,"Flowy maxi dress in earthy tones, tassels and ...",0.000000
2,Cozy Knit Sweater,"Soft oversized knit, neutral color, perfect fo...",0.000000


Good matches: 1 | Latency: 2.12 ms
--------------------------------------------------------------------------------
 Query: cozy hygge sweater


,name,desc,score
0,Cozy Knit Sweater,"Soft oversized knit, neutral color, perfect fo...",0.406007
1,Boho Dress,"Flowy maxi dress in earthy tones, tassels and ...",0.000000
2,Urban Bomber,"Cropped bomber jacket, sleek lines, reflective...",0.000000


Good matches: 1 | Latency: 2.18 ms
--------------------------------------------------------------------------------
 Query: evening glam slip dress


,name,desc,score
0,Silk Slip Dress,Bias-cut silk slip in jewel tones for evening ...,0.371976
1,Boho Dress,"Flowy maxi dress in earthy tones, tassels and ...",0.089749
2,Urban Bomber,"Cropped bomber jacket, sleek lines, reflective...",0.000000


Good matches: 1 | Latency: 2.6 ms
--------------------------------------------------------------------------------


,query,good_matches,latency_ms
0,energetic urban chic,1,2.1160
1,cozy hygge sweater,1,2.1849
2,evening glam slip dress,1,2.6012


In [46]:

reflection_points = [
    "1️⃣ Replaced paid API embeddings with free TF-IDF vectors — fast, lightweight, and no billing required.",
    "2️⃣ System returns top-3 fashion products for any 'vibe' query using cosine similarity.",
    "3️⃣ Handled edge cases — added fallback message if similarity < 0.3.",
    "4️⃣ Achieved average query latency under 5ms — very responsive for small datasets.",
    "5️⃣ Future upgrades: integrate OpenAI embeddings, Pinecone for large-scale vector search, and user-personalized recommendations."
]

print(' Reflection & Improvements:')
for line in reflection_points:
    print(line)


 Reflection & Improvements:
1️⃣ Replaced paid API embeddings with free TF-IDF vectors — fast, lightweight, and no billing required.
2️⃣ System returns top-3 fashion products for any 'vibe' query using cosine similarity.
3️⃣ Handled edge cases — added fallback message if similarity < 0.3.
4️⃣ Achieved average query latency under 5ms — very responsive for small datasets.
5️⃣ Future upgrades: integrate OpenAI embeddings, Pinecone for large-scale vector search, and user-personalized recommendations.
